## 1 Environment setup

### 1.1 Dependency

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from torchsummary import summary
import time

### 1.2 Connect to machine

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## 2 Functions

### 2.1 Generate training samples

In [ ]:
def sample(N):
    global x
    x = torch.zeros([N, N, 2]).to(device)
    for i in range(N):
        for j in range(N):
            x[i][j][0] = i; # x
            x[i][j][1] = j; # y
    x.requires_grad = True

### 2.2 Generate benchmark solution

In [ ]:
def compute_FD(N, tol = 1e-6):
    global Tn
    Tn = np.zeros((N, N))
    Tn[:, N - 1] = 1
    
    h = 1 / (N - 1)
    iteration = 0
    res = 1
    
    print("tol: ", tol)
    
    while(res > tol):
        iteration = iteration + 1
        res = 0
        for i in range(1, N - 1):
            for j in range(1, N - 1):
                prev = Tn[i, j]
                Tn[i, j] = Tn[i + 1 , j] + Tn[i - 1, j] + Tn[i, j + 1] + Tn[i, j - 1]
                Tn[i, j] = Tn[i, j] * 0.25
                res = max(res, np.abs((Tn[i, j] - prev) / (prev + 1e-20))) 
            
        print("iteration: ", iteration, "res: ", res)
        
    plt.subplots()[1].set_box_aspect(1)
    plt.contourf(np.transpose(Tn), vmax = Tn.max(), vmin = Tn.min(), levels = 20)
    plt.colorbar()
    
    
    fig, ax = plt.subplots(1, 1, figsize=(17,5))
    ax.plot(np.linspace(0, N - 1, N), Tn[:, int((N-1)/2)], '-o')
    ax.set(xlim=(0, N))
    ax.set_box_aspect(1)
    plt.xlabel('j')
    plt.ylabel('T')
    plt.legend(['FDM', 'PINN'])
    plt.xlim((0,N-1))
    return Tn 

### 2.3 PINN

##### Variables on the focus:
+ pl(neuron "p"er "l"ayer): 64 by default.
+ Depth of NN tunned by commenting out the forward function.
+ DAB("d"imension "a"nalysis "b"ased, i.e weighing scheme):
    0. loss weight: same weight | matrix: MSE
    1. loss weight: weighted by dimension analyzed result | matrix: MSE
    2. loss weight: weighted by root of dimension analyzed result | matrix: MSE
    3. loss weight: same weight | matrix: MAE
    4. loss weight: weighted by dimension analyzed result | matrix: MAE
    5. loss weight: weighted by root of dimension analyzed result | matrix: MAE

In [ ]:
def predict_NN(N, w_DE = 1, w_BC = 1, DAB = 0, start_epoch = 0, pl = 64, train_epoch = 50000):
    class Net(torch.nn.Module):
        def __init__(self, n_feature, n_hidden, n_hidden1, n_hidden2, n_hidden3, n_hidden4, n_hidden5, n_hidden6, n_hidden7, n_hidden8, n_hidden9, n_output):
            super(Net, self).__init__()
            self.hidden = torch.nn.Linear(n_feature, n_hidden)
            self.hidden1 = torch.nn.Linear(n_hidden, n_hidden1)
            self.hidden2 = torch.nn.Linear(n_hidden1, n_hidden2)
            self.hidden3 = torch.nn.Linear(n_hidden2, n_hidden3)
            self.hidden4 = torch.nn.Linear(n_hidden3, n_hidden4)
            self.hidden5 = torch.nn.Linear(n_hidden4, n_hidden5)
            self.hidden6 = torch.nn.Linear(n_hidden5, n_hidden6)
            self.hidden7 = torch.nn.Linear(n_hidden6, n_hidden7)
            self.hidden8 = torch.nn.Linear(n_hidden7, n_hidden8)
            self.hidden9 = torch.nn.Linear(n_hidden8, n_hidden9)
            self.hidden10 = torch.nn.Linear(n_hidden8, n_hidden9)
            self.hidden11 = torch.nn.Linear(n_hidden8, n_hidden9)
            self.hidden12 = torch.nn.Linear(n_hidden8, n_hidden9)
            self.hidden13 = torch.nn.Linear(n_hidden8, n_hidden9)
            self.predict = torch.nn.Linear(n_hidden9, n_output)

        def forward(self, x):
            x = torch.sin(self.hidden(x))
            x = torch.sin(self.hidden1(x))
            x = torch.sin(self.hidden2(x))
            x = torch.sin(self.hidden3(x))
            
#             x = torch.sin(self.hidden4(x))
#             x = torch.sin(self.hidden5(x))
#             x = torch.sin(self.hidden6(x))
#             x = torch.sin(self.hidden7(x))
#             x = torch.sin(self.hidden8(x))

#             x = torch.sin(self.hidden9(x))
#             x = torch.sin(self.hidden10(x))
#             x = torch.sin(self.hidden11(x))
#             x = torch.sin(self.hidden12(x))
#             x = torch.sin(self.hidden13(x))
            
            x = self.predict(x)
            return x
    

    net = Net(n_feature=2, n_hidden = pl, n_hidden1 = pl, n_hidden2 = pl, n_hidden3 = pl, n_hidden4 = pl, n_hidden5 = pl, n_hidden6 = pl, n_hidden7 = pl, n_hidden8 = pl, n_hidden9 = pl, n_output=1)     # define the network
    net.to(device)
    summary(net, (1,1, 2))

    ########

    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    
        
    loss_func = torch.nn.MSELoss()
    if (DAB == 3)|(DAB == 4)|(DAB == 5):
        loss_func = torch.nn.L1Loss()
        
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.8)
    plt.ion()

    ########

    if(start_epoch == 0):
        loss_epoch=[]
        loss_values = []
        loss_DE_values = []
        loss_DBC_values = []

    h = 1 / (N - 1)
    loss_op = 'mean'


    ###############################################################
    # Define loss weight                                          #
    ###############################################################

    
    if DAB == 0: #MSE(1)
        b = w_DE
        a = w_BC
    elif DAB == 1: #MSE(DA)
        b = w_DE /((N-1)*(N-1)*(N-1)*(N-1))
        a = w_BC 
    elif DAB == 2: #MSE(Weak)
        b = w_DE /((N-1)*(N-1))
        a = w_BC 
    elif DAB == 3: #MAE(1)
        b = w_DE
        a = w_BC 
    elif DAB == 4: #MAE(DA)
        b = w_DE/((N-1)*(N-1))
        a = w_BC 
    elif DAB == 5: #MAE(Weak)
        b = w_DE /(N-1)
        a = w_BC 
    
    ###############################################################
    # Loss weight normalization                                   #
    ###############################################################
    
    print(b, a)
    
    lambda_DE = b / (a + b)
    lambda_DBC = a / (a + b)


    ###############################################################
    # Training                                                    #
    ###############################################################

    start = time.perf_counter()
    for t in range(start_epoch, train_epoch):
    
        scheduler.step()
        T_out = net(x)
        T = T_out[:, :, 0]

        lap_T = (-4 * T[1:-1, 1:-1] + T[2:, 1:-1] + T[:-2, 1:-1] + T[1:-1, 2:] + T[1:-1, :-2]) / (h * h)
        loss_DE = loss_func(lap_T , torch.zeros((N - 2 , N - 2)).to(device))
        loss_DBC = loss_func(T[:, 0], torch.zeros((N , 1)).to(device)) #s
        loss_DBC = loss_DBC + loss_func(T[:, -1] - 1, torch.zeros((N , 1)).to(device)) #n
        loss_DBC = loss_DBC + loss_func(T[0, :], torch.zeros((1 , N)).to(device)) #w
        loss_DBC = loss_DBC + loss_func(T[-1, :], torch.zeros((1 , N)).to(device)) #e

        loss =  lambda_DE * loss_DE + lambda_DBC * loss_DBC

        loss_epoch.append(t)
        loss_DE_values.append(loss_DE.item())
        loss_DBC_values.append(loss_DBC.item())
        loss_values.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if t % 1000 == 0:
            print("Epoch: ", t, " ,loss: ", loss)
            print("loss_DE: ", loss_DE)
            print("loss_DBC(*lambda_DBC): ", loss_DBC)
            print("Timer: {0:.2f} sec".format(time.perf_counter() - start)) 
            plt.subplots()[1].set_box_aspect(1)   
            plt.cla()
            plt.contourf(np.transpose(torch.reshape(T, (N, N)).cpu().detach().numpy()), vmax = T.max(), vmin = T.min())
            plt.colorbar()
            plt.pause(0.1)     

    #####
    
    print("Epoch: ", t, " ,loss: ", loss)
    plt.subplots()[1].set_box_aspect(1)  
    plt.cla()
    plt.contourf(np.transpose(torch.reshape(T, (N, N)).cpu().detach().numpy()), vmax = Tn.max(), vmin = Tn.min(), levels = 20)
    plt.colorbar()
    plt.pause(0.1)

    plt.ioff()
    plt.show()
    
    #####
    
    print("Loss curve: ")
    plt.plot(loss_epoch, loss_values, loss_epoch, loss_DE_values, loss_epoch, loss_DBC_values)
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend(['$\mathcal{L}$', '$\mathcal{L}_{DE}$','$\mathcal{L}_{DBC}$'])
    plt.semilogy()
    plt.pause(0.1)
    
    #####
    
    print("Error: ")
    plt.subplots()[1].set_box_aspect(1)  
    plt.cla()
    plt.contourf(np.transpose(torch.reshape(T, (N, N)).cpu().detach().numpy() - Tn), vmax = Tn.max(), vmin = Tn.min(), levels = 20)
    plt.colorbar()
    plt.pause(0.1)

    plt.ioff()
    plt.show()
    
    #####
    
    print("y = 0.5: ")
    fig, ax = plt.subplots(1, 1, figsize=(17,5))
    plot_x = np.linspace(0, N - 1, N)
    ax.plot(plot_x, np.transpose(Tn[:, int((N-1)/2)]), plot_x, T[:, int((N-1)/2)].cpu().detach().numpy(), 'r--')
    
    ax.set(xlim=(0, N - 1))
    ax.set_box_aspect(1)
    plt.xlabel('j')
    plt.ylabel('T')
    plt.legend(['Ground truth', 'PINN'])
    plt.pause(0.1)
    
    #####
    
    pred = Tn
    exact = torch.reshape(T, (N, N)).cpu().detach().numpy()
    mse = np.mean(np.power((pred - exact), 2))
    
    pred_1 = Tn[:, int((N-1)/2)]
    exact_1 = torch.reshape(T, (N, N)).cpu().detach().numpy()[:, int((N-1)/2)]
    mse_1 = np.mean(np.power((pred_1 - exact_1), 2))

    print(f" ||||| {loss.item() :.4} | {loss_DE.item() :.4} | {loss_DBC.item() :.4} | {time.perf_counter() - start :.2f} sec | {mse :.4} | {mse_1 :.4} |")
    return (torch.reshape(T, (N, N)).cpu().detach().numpy())

### 2.4 Post process

In [ ]:
def plotmid(Tn = np.zeros((11, 11)), T1 = np.zeros((11, 11)), T2 = np.zeros((11, 11)), Tx = np.zeros((11, 11)), N = 11):
# def plotmid(Tn = np.zeros((11, 11)), T1 = np.zeros((11, 11)), T2 = np.zeros((11, 11)), N = 11):
    print("x = 0.5: ")
    fig, ax = plt.subplots(figsize=(10,10))
    plot_x = np.linspace(0, 1, N)
    ax.plot(plot_x, (Tn[:, int((N-1)/2)]), '-k', markerfacecolor='none')
    ax.plot(plot_x, T1[:, int((N-1)/2)], 'sr', ms = 10, markerfacecolor='none')
    ax.plot(plot_x, T2[:, int((N-1)/2)], '^b', ms = 10, markerfacecolor='none')
    ax.plot(plot_x, Tx[:, int((N-1)/2)], '+g', ms = 10, markerfacecolor='none')

    ax.set(xlim=(0, 1))
    ax.set_box_aspect(1)
    plt.xlabel('y')
    plt.ylabel('T')
    plt.legend(['$T_{FDM}$', '$\hatT_{1}$', '$\hatT_{2}$', '$\hatT_{3}$'], fontsize="14", frameon=False)
    plt.pause(0.1)

## 3. Experiment

In [ ]:
###############################################################
# Sample                                                      #
###############################################################

In [ ]:
N = 101

In [ ]:
Tn = compute_FD(N, tol= 1e-2) 

In [ ]:
sample(N)

In [ ]:
Ta = predict_NN(N, pl = 64, DAB = 2, train_epoch = 1000) #101 64*5

In [ ]:
Tb = predict_NN(N, pl = 128, DAB = 2, train_epoch = 1000) #101 128*5

In [ ]:
Tc = predict_NN(N, pl = 192, DAB = 2, train_epoch = 1000) #101 192*5

In [ ]:
plotmid(Tn, Ta, Tb, Tc, N = N) #101 64*5 vs 128*5 vs 192*5